<a href="https://colab.research.google.com/github/nenecchiCV/internal-competition-2024/blob/main/data_aug1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
api_key = "b846546e2316d4194c74a497599a7b83c9127f07"

In [3]:
!pip install colorama
!pip install sdv
!pip install wandb -qqq
import wandb

try:
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.2/143.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import pandas as pd
import numpy as np
data1_df = pd.read_csv('/content/drive/MyDrive/daicc2024/data_1.csv')
data2_df = pd.read_csv('/content/drive/MyDrive/daicc2024/data_2.csv')
data_df = pd.concat([data1_df, data2_df], axis=0).drop(columns=["id"])



test_df = pd.read_csv('/content/drive/MyDrive/daicc2024/test.csv')
all_df = pd.concat([data_df, test_df], axis=0)

data_df.head()

,obs_11,obs_12,obs_13,obs_14,obs_15,obs_21,obs_22,obs_23,obs_24,obs_25,obs_31,obs_32,obs_33,obs_34,obs_35,parameter,spec
0,62.01457,91.65045,48.20859,73.31024,76.13590,60.20358,96.17887,87.30840,84.63313,40.50873,70.01471,14.47043,42.81627,50.29044,113.68448,0.0,98.64420
1,60.69702,90.91513,48.58664,73.18577,76.88932,61.62932,98.17003,88.02719,83.83372,43.05050,53.33085,24.38429,104.27514,36.71392,104.62515,0.0,100.92119
2,60.47336,90.45913,48.65129,73.11828,76.60147,61.97207,97.29511,85.52639,86.61886,40.13236,76.22214,13.96032,29.86299,60.08177,119.78403,0.0,97.04798
3,60.56826,90.90713,48.94006,73.18156,77.07093,60.86407,96.14732,87.48353,84.64801,40.40228,73.93093,18.26402,60.49384,52.88122,114.96104,0.0,96.81147
4,60.63285,90.73787,48.52429,73.19398,76.66164,59.77490,96.68002,87.39572,86.15437,40.13888,63.38943,29.94443,78.32125,46.80220,103.40444,0.0,100.10177


メモ：計測値とスコアをX、パラメータをYとして学習→結果を出すときはスコアを100にする

RNN
https://qiita.com/MENDY/items/99da56f61f9af51dda15

In [5]:
from torch.utils import data as data

class MyDataset(data.Dataset):
    def __init__(self, X1_scaled_tensor, x2_scaled_tensor, y_tensor):
        self.X1 = X1_scaled_tensor
        self.x2 = x2_scaled_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return self.X1[index], self.x2[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [6]:
reasonable_spec_constraint = {
    'constraint_class': 'ScalarRange',
    'constraint_parameters': {
        'column_name': 'spec',
        'low_value': 90-0.00000001,
        'high_value': 110+0.00000001
    }
}

reasonable_parameter_constraint = {
    'constraint_class': 'ScalarRange',
    'constraint_parameters': {
        'column_name': 'parameter',
        'low_value': -5-0.00000001,
        'high_value': 5+0.00000001
    }
}

In [8]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data_df)

constraints = [
               reasonable_spec_constraint,
               reasonable_parameter_constraint
]


synthesizer = CTGANSynthesizer(metadata=metadata, epochs=1000, verbose=True)
synthesizer.add_constraints(
    constraints=constraints
)
synthesizer.fit(data_df)



/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:79: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Gen. (-1.69) | Discrim. (-0.04): 100%|██████████| 1000/1000 [17:52<00:00,  1.07s/it]
Sampling rows: 100%|██████████| 20000/20000 [00:00<00:00, 26238.79it/s]


In [14]:
synthetic_data = synthesizer.sample(num_rows=30000)

Sampling rows: 100%|██████████| 30000/30000 [00:01<00:00, 27614.19it/s]


In [18]:
synthetic_data["parameter"].unique()

array([ 0. ,  0.7,  0.8, -2.2, -0.5,  4.3,  1.6, -1.5, -2.1, -3.3,  0.6,
       -1. ,  2.6,  2.5,  4.5,  0.9, -2.3, -2.7,  2.8, -2. , -2.6,  4.2,
        4.6,  1.7,  1.5, -3.1, -2.8,  1.4,  3.8, -3.2,  3.4, -3.4,  1.3,
        2.9,  2.7,  3.7,  1.8,  4.1,  3. , -2.9,  1.9,  1.1,  3.1,  3.9,
       -2.5,  4. ,  4.7, -3. ,  2.4,  2.3, -1.4, -1.9, -3.5,  3.6,  1.2,
        4.4,  0.5, -2.4,  3.2,  2.1,  3.5,  0.4, -1.8, -3.6,  2. ,  1. ,
        3.3,  2.2,  4.8, -0.9, -3.7, -3.8])

In [11]:
from sdv.evaluation.single_table import evaluate_quality
from sdv.evaluation.single_table import run_diagnostic

diagnostic_report = run_diagnostic(
    real_data=data_df,
    synthetic_data=synthetic_data,
    metadata=metadata)

quality_report = evaluate_quality(
    real_data=data_df,
    synthetic_data=synthetic_data,
    metadata=metadata)

Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<00:00, 188.43it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 136/136 [00:02<00:00, 59.80it/s]

Overall Score: 93.92%

Properties:
- Column Shapes: 91.57%
- Column Pair Trends: 96.26%


In [13]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=data_df,
    synthetic_data=synthetic_data,
    metadata=metadata,
    column_name='spec'
)

fig.show()

In [19]:
synthesizer.save(
    filepath='/content/drive/MyDrive/daicc2024/my_synthesizer.pkl'
)
metadata.save_to_json(filepath='/content/drive/MyDrive/daicc2024/my_metadata_v1.json')

In [20]:
synthetic_data.to_csv(f'/content/drive/MyDrive/daicc2024/data_aug.csv')

In [ ]:
# from sdv.single_table import CTGANSynthesizer

# synthesizer = CTGANSynthesizer.load(
#     filepath='/content/drive/MyDrive/daicc2024/my_synthesizer.pkl'
# )

TabDDPMというDiffusion modelの手法もある

https://github.com/yandex-research/tab-ddpm

使えなかった